# WPC 만들기

In [ ]:
import pandas as pd

# 1. 엑셀 파일 경로
file_path = ".xlsx"  # 필요시 경로 수정

# 2. 시트 불러오기
df_work = pd.read_excel(file_path, sheet_name="Work")
df_pub = pd.read_excel(file_path, sheet_name="Publication")
df_col = pd.read_excel(file_path, sheet_name="Collection")
df_edge_pub_w = pd.read_excel(file_path, sheet_name="Edge_(Pub-W)")
df_edge_pub_c = pd.read_excel(file_path, sheet_name="Edge_(Pub-C)")

# 3. 병합용 원본 ID 보존
df_work["work_id_orig"] = df_work["id"]
df_pub["pub_id_orig"] = df_pub["id"]
df_col["col_id_orig"] = df_col["id"]

# 4. 컬럼 prefix 부여
df_work_prefixed = df_work.rename(columns=lambda x: f"work_{x}" if x != "work_id_orig" else x)
df_pub_prefixed = df_pub.rename(columns=lambda x: f"pub_{x}" if x != "pub_id_orig" else x)
df_col_prefixed = df_col.rename(columns=lambda x: f"col_{x}" if x != "col_id_orig" else x)

# 5. 엣지 병합
edge_pub_w = df_edge_pub_w.rename(columns={"source id": "pub_id", "target id": "work_id"})
edge_pub_c = df_edge_pub_c.rename(columns={"source id": "pub_id", "target id": "col_id"})
pub_core = pd.merge(edge_pub_c, edge_pub_w, on="pub_id", how="outer")

# 6. 정보 병합 (ID 기준, prefix 이전 ID 사용)
pub_core = pub_core.merge(df_pub_prefixed, left_on="pub_id", right_on="pub_id_orig", how="left")
pub_core = pub_core.merge(df_work_prefixed, left_on="work_id", right_on="work_id_orig", how="left")
pub_core = pub_core.merge(df_col_prefixed, left_on="col_id", right_on="col_id_orig", how="left")

# 7. 연결된 ID 목록 추출
linked_work_ids = pub_core['work_id_x'].dropna().unique().tolist()
linked_pub_ids = pub_core['pub_id_x'].dropna().unique().tolist()
linked_col_ids = pub_core['col_id_x'].dropna().unique().tolist()
# 8. 단독 항목 추출
solo_work = df_work_prefixed[~df_work_prefixed['work_id_orig'].isin(linked_work_ids)].copy()
solo_pub = df_pub_prefixed[~df_pub_prefixed['pub_id_orig'].isin(linked_pub_ids)].copy()
solo_col = df_col_prefixed[~df_col_prefixed['col_id_orig'].isin(linked_col_ids)].copy()

# 9. 최종 출력 컬럼 정의
final_columns = [
    'col_id_orig', 'col_name',
    'pub_id_orig', 'pub_name',
    'work_id_orig', 'work_korname', 'work_chiname',
    'work_titleExam', 'work_style', 'work_original', 'work_translation', 'work_url'
]

# 10. 누락 컬럼 보완 및 정렬
for df in [solo_work, solo_pub, solo_col]:
    for col in final_columns:
        if col not in df.columns:
            df[col] = None
    df = df[final_columns]

pub_core_final = pub_core[final_columns]

# 11. 전체 병합
full_merged = pd.concat([pub_core_final, solo_work, solo_pub, solo_col], ignore_index=True)

# 12. 저장
full_merged.to_csv("full_merged_with_prefix.csv", index=False)

## EPQA생성
### 1. P-A결합

In [ ]:
import pandas as pd

# === 경로 설정 ===
epqa_path = "EPQA.xlsx"

# === 시트 로드 (컬럼명 정정 반영)
df_person = pd.read_excel(epqa_path, sheet_name="Person").rename(columns={"id": "person_id"})
df_answer = pd.read_excel(epqa_path, sheet_name="Answer").rename(columns={"id": "answer_id"})  # writer는 그대로 사용
df_edge_pea = pd.read_excel(epqa_path, sheet_name="Edge_(Pe-A)").rename(
    columns={"source id": "person_id", "target id": "answer_id"}
).dropna().drop_duplicates()

# === Step 1: PA 병합 (단독 A, P 포함)
# answer 기준 병합
df_pa_left = df_answer.merge(df_edge_pea, on="answer_id", how="left") \
                      .merge(df_person, on="person_id", how="left")

# person 기준 병합
df_pa_right = df_person.merge(df_edge_pea, on="person_id", how="left") \
                       .merge(df_answer, on="answer_id", how="left")

# 병합 결과 통합 + 중복 제거
df_pa_all = pd.concat([df_pa_left, df_pa_right], ignore_index=True).drop_duplicates(subset=["person_id", "answer_id"])

# === 작성자 일치 여부 플래그 추가 (writer vs fullname)
def check_writer_match(row):
    try:
        return pd.notna(row["writer"]) and pd.notna(row["fullname"]) and row["writer"].strip() in row["fullname"]
    except:
        return False

df_pa_all["is_writer_match"] = df_pa_all.apply(check_writer_match, axis=1)

# === 저장: 전체, 일치, 불일치 시트 분리
with pd.ExcelWriter("step1_PA_flagged.xlsx") as writer:
    df_pa_all.to_excel(writer, index=False, sheet_name="PA_ALL")
    df_pa_all[df_pa_all["is_writer_match"] == True].to_excel(writer, index=False, sheet_name="PA_Match")
    df_pa_all[df_pa_all["is_writer_match"] == False].to_excel(writer, index=False, sheet_name="PA_Mismatch")

print("✅ Step 1 (PA) 저장 완료: 전체 {}, 일치 {}, 불일치 {}".format(
    len(df_pa_all),
    df_pa_all["is_writer_match"].sum(),
    len(df_pa_all) - df_pa_all["is_writer_match"].sum()
))

### 2. (P-A)Q결합

In [3]:
import pandas as pd

# === 1. 파일 경로
pea_path = "EPQA_PeA_with_edge_hit.xlsx"
epqa2_path = "EPQA2.xlsx"

# === 2. Pe-A 결합 결과 불러오기 (edge_hit1 포함)
df_pea = pd.read_excel(pea_path)
df_pea = df_pea.rename(columns={"edge_hit": "edge_hit1"})

# === 3. A-Q 엣지 및 질문 시트 불러오기 (name 기준)
df_edge_aq = pd.read_excel(epqa2_path, sheet_name="Edge_(A-Q)").rename(
    columns={"source name": "answer_name", "target name": "question_name"}
).dropna().drop_duplicates()

df_question = pd.read_excel(epqa2_path, sheet_name="Question").rename(
    columns={"name": "question_name", "id": "question_id"}
)

# === 4. 병합: A-Q 연결 정보 붙이기
df_paq = df_pea.merge(df_edge_aq, how="left", on="answer_name") \
               .merge(df_question, how="left", on="question_name")

# === 5. edge_hit2 추가: 정밀 매칭 여부 판단
df_edge_aq["key"] = df_edge_aq["answer_name"].astype(str) + "|" + df_edge_aq["question_name"].astype(str)
df_paq["key"] = df_paq["answer_name"].astype(str) + "|" + df_paq["question_name"].astype(str)
df_paq["edge_hit2"] = df_paq["key"].isin(df_edge_aq["key"])
df_paq = df_paq.drop(columns=["key"])

# === 6. 저장
df_paq.to_excel("step2_PeAQ_with_edge_hit2.xlsx", index=False)

print("✅ 완료: 전체 {}, A-Q 연결 수 (edge_hit2=True): {}".format(
    len(df_paq),
    df_paq["edge_hit2"].sum()
))

✅ 완료: 전체 498, A-Q 연결 수 (edge_hit2=True): 346


### 3. (P-A-Q)E 결합

In [ ]:
import pandas as pd

# === 1. 경로 설정
step2_path = "step2_PeAQ_with_edge_hit2.xlsx"
epqa2_path = "EPQA2.xlsx"

# === 2. 이전 병합 파일 불러오기 (edge_hit1, edge_hit2 포함)
df_paq = pd.read_excel(step2_path)

# === 3. Q-E 엣지 및 Exam 시트 불러오기
df_edge_qe = pd.read_excel(epqa2_path, sheet_name="Edge_(Q-E)").rename(
    columns={"source name": "question_name", "target name": "exam_name"}
).dropna().drop_duplicates()

df_exam = pd.read_excel(epqa2_path, sheet_name="Exam").rename(
    columns={"name": "exam_name", "id": "exam_id"}
)

# === 4. 병합: Q-E 연결 및 Exam 정보
df_paqe = df_paq.merge(df_edge_qe, how="left", on="question_name") \
                .merge(df_exam, how="left", on="exam_name")

# === 5. edge_hit3 계산 (정확히 연결된 Q-E 쌍에 대해서)
df_edge_qe["key"] = df_edge_qe["question_name"].astype(str) + "|" + df_edge_qe["exam_name"].astype(str)
df_paqe["key"] = df_paqe["question_name"].astype(str) + "|" + df_paqe["exam_name"].astype(str)
df_paqe["edge_hit3"] = df_paqe["key"].isin(df_edge_qe["key"])
df_paqe = df_paqe.drop(columns=["key"])

# === 6. 저장
df_paqe.to_excel("step3_PeAQE_with_edge_hit3.xlsx", index=False)

print("✅ 완료: 전체 {}, Q-E 연결 수 (edge_hit3=True): {}".format(
    len(df_paqe),
    df_paqe["edge_hit3"].sum()
))

=== Step 4: Answer-Work 합집합 병합 시작 ===
원본 PeAQE 데이터: 1734 행
유효한 A-W 엣지: 272 개
Work 데이터: 865 개
\n=== 합집합 병합 수행 ===
Step 4-1: PeAQE + Edge = 10443 행
  - Edge 매칭됨: 10193 행
  - Edge 없음: 250 행
Step 4-2: 최종 병합 = 52901 행
  - Work 매칭됨: 42924 행
  - Work 없음: 9977 행
\n=== edge_hit4 플래그 결과 ===
edge_hit4=True (A∩W): 424 행
edge_hit4=False (A-W): 52477 행
총합: 52901 행
\n=== 최종 데이터 품질 검증 ===
Answer 데이터 보존: 1598 → 29781
Work 데이터 추가: 424 행
Work 없는 Answer: 52477 행
✅ 모든 Answer 데이터 보존됨
\n=== 중복 제거 ===
중복 제거: 52901 → 224 행 (-52677)
\n결과 저장: step4_PeAQEW_union_with_edge_hit4.xlsx
\n==================================================
합집합 병합 완료!
- 전체 데이터: 224 행
- Answer+Work: 4 행 (교집합)
- Answer만: 220 행 (차집합)
- 매칭률: 1.8%


### 4. CPW-EPQA 연결

In [2]:
import pandas as pd
import numpy as np

def extract_edge_matches_only():
    """
    Edge_(A-W) 매칭만 추출 + 원본 파일들에 hit 플래그 추가
    
    출력:
    1. edge_matched_only.xlsx - 292개 매칭 데이터만
    2. step3_with_flag.xlsx - 원본 + edge_hit_flag 컬럼
    3. cpw_work_with_flag.xlsx - 원본 + edge_hit_flag 컬럼
    """
    
    print("=== Edge 매칭 추출 및 플래그 생성 ===")
    
    # === Step 1: 파일 로드 ===
    step3_path = "step3_PeAQE_with_edge_hit3.xlsx"
    cpw_path = "CPW.xlsx"
    
    df_step3 = pd.read_excel(step3_path, sheet_name="Sheet1")
    df_edge_raw = pd.read_excel(cpw_path, sheet_name="Edge_(A-W)")
    df_work_raw = pd.read_excel(cpw_path, sheet_name="Sheet2")
    
    print(f"📁 Step3: {len(df_step3)} 행")
    print(f"📁 Edge 원본: {len(df_edge_raw)} 행")
    print(f"📁 Work 원본: {len(df_work_raw)} 행")
    
    # === Step 2: Edge 데이터 정제 ===
    print("\\n=== Step 2: Edge 데이터 정제 ===")
    
    # 헤더 행 제거 및 유효 데이터만 필터링
    df_edge = df_edge_raw[
        (df_edge_raw['source id'] != '소스 식별자') &
        (df_edge_raw['source id'].notna()) &
        (df_edge_raw['target id'].notna())
    ].copy()
    
    # 중복 제거
    df_edge = df_edge.drop_duplicates(
        subset=['source id', 'target id']
    ).reset_index(drop=True)
    
    print(f"🧹 Edge 정제: {len(df_edge_raw)} → {len(df_edge)} 행")
    
    # Work 데이터 정제
    df_work = df_work_raw[
        df_work_raw['work_id_orig'].notna()
    ].drop_duplicates(
        subset=['work_id_orig'], keep='first'
    ).reset_index(drop=True)
    
    print(f"🧹 Work 정제: {len(df_work_raw)} → {len(df_work)} 행")
    
    # === Step 3: 매칭 가능한 Edge만 필터링 ===
    print("\\n=== Step 3: 매칭 가능한 Edge 필터링 ===")
    
    # Step3와 매칭 가능한 source id
    step3_source_ids = set(df_step3['source id_x'].dropna())
    edge_matchable_step3 = df_edge[
        df_edge['source id'].isin(step3_source_ids)
    ].copy()
    
    print(f"🔗 Step3 매칭 가능: {len(edge_matchable_step3)}/{len(df_edge)}")
    
    # Work와 매칭 가능한 target id
    work_ids = set(df_work['work_id_orig'])
    edge_matchable_both = edge_matchable_step3[
        edge_matchable_step3['target id'].isin(work_ids)
    ].copy()
    
    print(f"🔗 양쪽 모두 매칭 가능: {len(edge_matchable_both)}")
    
    # === Step 4: Edge 매칭 데이터만 추출 ===
    print("\\n=== Step 4: Edge 매칭 데이터 추출 ===")
    
    # 4-0: Edge 중복 키 문제 해결
    print("🔍 Edge 중복 키 확인...")
    edge_dup_check = edge_matchable_both['source id'].value_counts()
    duplicated_sources = edge_dup_check[edge_dup_check > 1]
    
    if len(duplicated_sources) > 0:
        print(f"⚠️  중복 source id 발견: {len(duplicated_sources)}개")
        print("예시:", duplicated_sources.head(3).to_dict())
        
        # 중복 제거: source id별로 첫 번째만 유지
        edge_dedup = edge_matchable_both.drop_duplicates(
            subset=['source id'], keep='first'
        ).reset_index(drop=True)
        
        print(f"🧹 중복 제거: {len(edge_matchable_both)} → {len(edge_dedup)}")
        edge_for_merge = edge_dedup
    else:
        print("✅ 중복 없음")
        edge_for_merge = edge_matchable_both
    
    # 4-1: Step3 데이터 추출 (Edge와 매칭되는 것만)
    df_step3_matched = df_step3.merge(
        edge_for_merge[['source id', 'target id', 'source name', 'target name']],
        left_on='source id_x',
        right_on='source id',
        how='inner',  # 매칭되는 것만
        validate='m:1'  # 이제 안전해야 함
    )
    
    print(f"📊 Step3 매칭 추출: {len(df_step3_matched)} 행")
    
    # 4-2: Work 중복 키 확인 및 해결
    work_dup_check = df_work['work_id_orig'].value_counts()
    duplicated_works = work_dup_check[work_dup_check > 1]
    
    if len(duplicated_works) > 0:
        print(f"⚠️  중복 work_id_orig 발견: {len(duplicated_works)}개")
        df_work = df_work.drop_duplicates(
            subset=['work_id_orig'], keep='first'
        ).reset_index(drop=True)
        print(f"🧹 Work 중복 제거 완료")
    
    # 4-3: Work 데이터 추가
    df_edge_complete = df_step3_matched.merge(
        df_work,
        left_on='target id',
        right_on='work_id_orig',
        how='inner',  # 완전 매칭만
        validate='m:1',
        suffixes=('', '_work')
    )
    
    print(f"📊 완전 매칭 추출: {len(df_edge_complete)} 행")
    
    # edge_hit4 플래그 (모두 True)
    df_edge_complete['edge_hit4'] = True
    
    # === Step 5: 원본 파일들에 플래그 추가 ===
    print("\\n=== Step 5: 원본 파일에 플래그 추가 ===")
    
    # 5-1: Step3에 플래그 추가
    df_step3_flagged = df_step3.copy()
    
    # step3의 각 행이 edge에 매칭되는지 확인 (중복 제거된 edge 사용)
    matched_source_ids = set(edge_for_merge['source id'])
    df_step3_flagged['edge_hit_flag'] = df_step3_flagged['source id_x'].isin(matched_source_ids)
    
    step3_hit_count = df_step3_flagged['edge_hit_flag'].sum()
    print(f"📌 Step3 플래그: {step3_hit_count}/{len(df_step3_flagged)} 행이 매칭됨")
    
    # 5-2: Work에 플래그 추가
    df_work_flagged = df_work.copy()
    
    # work의 각 행이 edge에 매칭되는지 확인 (중복 제거된 edge 사용)
    matched_target_ids = set(edge_for_merge['target id'])
    df_work_flagged['edge_hit_flag'] = df_work_flagged['work_id_orig'].isin(matched_target_ids)
    
    work_hit_count = df_work_flagged['edge_hit_flag'].sum()
    print(f"📌 Work 플래그: {work_hit_count}/{len(df_work_flagged)} 행이 매칭됨")
    
    # === Step 6: 저장 ===
    print("\\n=== Step 6: 파일 저장 ===")
    
    # 6-1: Edge 매칭만 저장
    edge_output = "edge_matched_only.xlsx"
    df_edge_complete.to_excel(edge_output, index=False)
    print(f"💾 Edge 매칭만: {edge_output} ({len(df_edge_complete)} 행)")
    
    # 6-2: Step3 + 플래그 저장
    step3_output = "step3_with_edge_flag.xlsx"
    df_step3_flagged.to_excel(step3_output, index=False)
    print(f"💾 Step3 + 플래그: {step3_output} ({len(df_step3_flagged)} 행)")
    
    # 6-3: Work + 플래그 저장
    work_output = "cpw_work_with_edge_flag.xlsx"
    df_work_flagged.to_excel(work_output, index=False)
    print(f"💾 Work + 플래그: {work_output} ({len(df_work_flagged)} 행)")
    
    # === Step 7: 결과 요약 ===
    print("\\n" + "="*60)
    print("🎉 Edge 매칭 추출 완료!")
    print()
    print("📁 생성된 파일들:")
    print(f"   1. {edge_output}: Edge 매칭 데이터만 ({len(df_edge_complete)} 행)")
    print(f"   2. {step3_output}: 원본 Step3 + 플래그 ({len(df_step3_flagged)} 행)")
    print(f"   3. {work_output}: 원본 Work + 플래그 ({len(df_work_flagged)} 행)")
    print()
    print("📊 매칭 통계:")
    print(f"   - Step3 매칭률: {step3_hit_count}/{len(df_step3_flagged)} ({step3_hit_count/len(df_step3_flagged)*100:.1f}%)")
    print(f"   - Work 매칭률: {work_hit_count}/{len(df_work_flagged)} ({work_hit_count/len(df_work_flagged)*100:.1f}%)")
    print(f"   - 완전 매칭: {len(df_edge_complete)} 행")
    print("="*60)
    
    # === Step 8: 샘플 데이터 확인 ===
    print("\\n🔍 Edge 매칭 샘플:")
    if len(df_edge_complete) > 0:
        sample = df_edge_complete.iloc[0]
        print(f"   answer_name: {sample.get('answer_name', 'N/A')}")
        print(f"   work_korname: {sample.get('work_korname', 'N/A')}")
        print(f"   work_style: {sample.get('work_style', 'N/A')}")
        print(f"   source id: {sample.get('source id', 'N/A')}")
        print(f"   target id: {sample.get('target id', 'N/A')}")
    
    return {
        'edge_matched': df_edge_complete,
        'step3_flagged': df_step3_flagged, 
        'work_flagged': df_work_flagged
    }

def create_summary_report(results):
    """
    결과 요약 보고서 생성
    """
    if results is None:
        return
    
    edge_df = results['edge_matched']
    step3_df = results['step3_flagged']
    work_df = results['work_flagged']
    
    print("\\n" + "="*50)
    print("📋 상세 분석 보고서")
    print("="*50)
    
    # Edge 매칭 분석
    print("\\n1. Edge 매칭 데이터 분석:")
    print(f"   총 매칭: {len(edge_df)} 행")
    
    if len(edge_df) > 0:
        # 문체별 분포
        if 'work_style' in edge_df.columns:
            style_dist = edge_df['work_style'].value_counts().head(5)
            print("   주요 문체:")
            for style, count in style_dist.items():
                print(f"     {style}: {count}개")
        
        # 연도별 분포
        if 'year' in edge_df.columns:
            year_dist = edge_df['year'].value_counts().head(5)
            print("   주요 연도:")
            for year, count in year_dist.items():
                print(f"     {year}: {count}개")
    
    # 플래그 분석
    print("\\n2. 플래그 분석:")
    step3_true = step3_df['edge_hit_flag'].sum()
    work_true = work_df['edge_hit_flag'].sum()
    
    print(f"   Step3: {step3_true}/{len(step3_df)} 매칭")
    print(f"   Work: {work_true}/{len(work_df)} 매칭")
    
    print("\\n✅ 작업 완료!")

# 실행
if __name__ == "__main__":
    results = extract_edge_matches_only()
    create_summary_report(results)

=== Edge 매칭 추출 및 플래그 생성 ===
📁 Step3: 243 행
📁 Edge 원본: 291 행
📁 Work 원본: 865 행
\n=== Step 2: Edge 데이터 정제 ===
🧹 Edge 정제: 291 → 279 행
🧹 Work 정제: 865 → 274 행
\n=== Step 3: 매칭 가능한 Edge 필터링 ===
🔗 Step3 매칭 가능: 228/279
🔗 양쪽 모두 매칭 가능: 228
\n=== Step 4: Edge 매칭 데이터 추출 ===
🔍 Edge 중복 키 확인...
⚠️  중복 source id 발견: 23개
예시: {'AS16790917': 6, 'AS16330000B': 4, 'AS15550301': 3}
🧹 중복 제거: 228 → 193
📊 Step3 매칭 추출: 229 행
📊 완전 매칭 추출: 229 행
\n=== Step 5: 원본 파일에 플래그 추가 ===
📌 Step3 플래그: 229/243 행이 매칭됨
📌 Work 플래그: 190/274 행이 매칭됨
\n=== Step 6: 파일 저장 ===
💾 Edge 매칭만: edge_matched_only.xlsx (229 행)
💾 Step3 + 플래그: step3_with_edge_flag.xlsx (243 행)
💾 Work + 플래그: cpw_work_with_edge_flag.xlsx (274 행)
\n============================================================
🎉 Edge 매칭 추출 완료!

📁 생성된 파일들:
   1. edge_matched_only.xlsx: Edge 매칭 데이터만 (229 행)
   2. step3_with_edge_flag.xlsx: 원본 Step3 + 플래그 (243 행)
   3. cpw_work_with_edge_flag.xlsx: 원본 Work + 플래그 (274 행)

📊 매칭 통계:
   - Step3 매칭률: 229/243 (94.2%)
   - Work 매칭률: 190/274 (69.